In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [2]:
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        #print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

In [3]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [4]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [14]:
def generate_summary(file_name, top_n=2):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)   
    print(" ")
    print(" ")
    is_noun = lambda pos: pos[:2] == 'NN'
    nouns = [word for (word, pos) in nltk.pos_tag(sentences[0]) if is_noun(pos)] 
    print(nouns)
    #for i in range(top_n):
        #summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    #print("Summarize Text: \n", ". ".join(summarize_text))

In [15]:
generate_summary( "artificial_intelligence.txt")

Indexes of top ranked_sentence order are  [(0.19251415795299892, ['Meanwhile,', 'the', 'Galaxy', 'S21', 'Ultra', 'is', 'what', 'the', 'original', 'Ultra', 'should', 'have', 'been', '-', 'jam-packed', 'with', 'high-end', 'features,', 'no', 'asterisks,', 'no', 'fine', 'print']), (0.15873015866938936, ['The', 'ultrawide', 'camera', 'had', 'a', 'great', 'sensor,', 'but', 'it', "couldn't", 'do', 'macro', 'shots', 'due', 'to', 'the', 'lack', 'of', 'autofocus']), (0.15873015866938936, ['The', 'telephoto', 'camera', 'was', 'advertised', 'as', '5x,', 'but', 'it', 'was,', 'in', 'fact,', '4x,', 'and', "let's", 'not', 'even', 'start', 'with', 'the', 'Space', 'Zoom']), (0.15154746149241677, ['Still', 'no', 'charger,', 'though.The', 'original', 'S20', 'Ultra', 'was', 'supposed', 'to', 'be', 'the', 'greatest', 'Galaxy', 'to', 'date,', 'but', 'it', 'fell', 'short', 'of', 'that']), (0.14542950761607093, ['This', 'year,', 'the', 'Galaxy', 'S21', 'series', 'came', 'earlier', 'than', 'expected,', 'and', '